## Recurrence, Depth and High-dimensional data
# Autoencoder notebook

In this notebook we introduce autoencoders, which perform unsupervised learning of the train set by forcing a compression of the latent space. 

*Please execute the cell bellow in order to initialize the notebook environment*

In [ ]:
%autosave 0
# %matplotlib inline
%matplotlib notebook

from __future__ import division, print_function
import numpy as np
import matplotlib.pyplot as plt
import keras
import mod3

plt.rcParams.update({'figure.figsize': (5.0, 4.0), 'lines.linewidth': 2.0})

## MNIST dataset import and pre-processing

*Please execute the cell bellow in order to prepare the MNIST dataset*

In [ ]:
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train[:, ::2, ::2].copy()
x_test = x_test[:, ::2, ::2].copy()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

x_shape = x_train.shape[1:]

print('train set shape:', x_train.shape)
print('test set shape:', x_test.shape)

## Simple autoencoder

In [ ]:
from keras.layers import Input, Dense
from keras.models import Model

n_epochs = 5
encoding_dim = 32

input_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
input_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

input_train_shape = input_train.shape[1]

input_layer = Input(shape=(input_train_shape,))
encoded = Dense(encoding_dim, activation='relu', name='encoded')(input_layer)
decoded = Dense(input_train_shape, activation='sigmoid')(encoded)

autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
autoencoder.summary()

history = autoencoder.fit(input_train, input_train,
                          epochs=n_epochs,
                          batch_size=32,
                          shuffle=True,
                          validation_data=(input_test, input_test))

output_test = autoencoder.predict(input_test)

plt.figure(figsize=(9, 2))
mod3.plot_generated(input_test, output_test, x_shape)

**EXPECTED OUTPUT**
```
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
input_1 (InputLayer)         (None, 196)               0         
_________________________________________________________________
encoded (Dense)              (None, 32)                6304      
_________________________________________________________________
dense_1 (Dense)              (None, 196)               6468      
=================================================================
Total params: 12,772.0
Trainable params: 12,772.0
Non-trainable params: 0.0
_________________________________________________________________
Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 5s - loss: 0.2410 - val_loss: 0.1783
Epoch 2/5
60000/60000 [==============================] - 6s - loss: 0.1590 - val_loss: 0.1417
Epoch 3/5
60000/60000 [==============================] - 5s - loss: 0.1324 - val_loss: 0.1221
Epoch 4/5
60000/60000 [==============================] - 5s - loss: 0.1174 - val_loss: 0.1108
Epoch 5/5
60000/60000 [==============================] - 5s - loss: 0.1084 - val_loss: 0.1039
```
<img src="fig/autoencoder_simple.png" style="width:90%;height:90%;display:inline;margin:1px">

## Denoising autoencoder

In [ ]:
n_epochs = 5
encoding_dim = 32
noise_factor = 0.2

input_train_noisy = input_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=input_train_shape) 
input_test_noisy = input_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=input_train_shape) 

input_train_noisy = np.clip(input_train_noisy, 0., 1.)
input_test_noisy = np.clip(input_test_noisy, 0., 1.)

input_layer = Input(shape=(input_train_shape,))
encoded = Dense(encoding_dim, activation='relu', name='encoded')(input_layer)
decoded = Dense(input_train_shape, activation='sigmoid')(encoded)

autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

history = autoencoder.fit(input_train_noisy, input_train,
                          epochs=n_epochs)

output_test = autoencoder.predict(input_test)

plt.figure(figsize=(9, 2))
mod3.plot_generated(input_test_noisy, output_test, x_shape)

**EXPECTED OUTPUT**
```
Epoch 1/5
60000/60000 [==============================] - 5s - loss: 0.2355     
Epoch 2/5
60000/60000 [==============================] - 4s - loss: 0.1583     
Epoch 3/5
60000/60000 [==============================] - 4s - loss: 0.1335     
Epoch 4/5
60000/60000 [==============================] - 4s - loss: 0.1198     
Epoch 5/5
60000/60000 [==============================] - 5s - loss: 0.1116
```
<img src="fig/autoencoder_denoise.png" style="width:90%;height:90%;display:inline;margin:1px">

## Extended exercises

**EXTENDED EXERCISE 1**

How does the size of the bottleneck layer affect the effectiveness of the denoising autoencoder? For instance, compare the output to the input images, and investigate the effects of noise magnitude.

**EXTENDED EXERCISE 2**

The basic autoencoder model presented in this notebook performs compression of the input space due to the smaller dimension of the bottleneck layer.

How does this compression compare to popular methods of dimensionality reduction such as PCA.